# 导入基本模块

In [1]:
import pandas as pd

import os
from os import path

In [31]:
import warnings

warnings.filterwarnings(action='ignore')

In [30]:
help(warnings.filterwarnings)

Help on function filterwarnings in module warnings:

filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)
    Insert an entry into the list of warnings filters (at the front).
    
    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters



# 定义函数

In [83]:
def get_frequency(data1, treshold = 1):
    '''
    data1.index.week表示一年中第几个星期。比如1月2日表示第1周。12月是50多周
    可以将所处周相同的。作为新的一个特征，然后使用groupby进行分类。统计周出现的频率。月出现的频率
    '''
    data1['Frequency'] = data1.imsi_.map(data1.imsi_.value_counts()) # 频率使用的是imsi_出现的次数
    data1 = data1[data1.Frequency >= treshold]
    data1.index = pd.to_datetime(data1.update_time_)
    data1['day'] = data1.index.day # 要统计一个imsi_每天，每周，每年出现的次数，可以增加新的特征列
    data1['week'] = data1.index.week
    data1['month'] = data1.index.month
    day = data1.groupby(['imsi_',data1.day])['device_id_'].value_counts().unstack(fill_value=0)
    week = data1.groupby(['imsi_',data1.week])['device_id_'].value_counts().unstack(fill_value=0)
    month = data1.groupby(['imsi_',data1.month])['device_id_'].value_counts().unstack(fill_value=0)
    print('data processed successfully')
    return day, week, month

In [84]:
def get_data(file_name):
    '''
    传入的是文件路径
    读取并对数据进行处理
    '''
    file_extension = path.splitext(file_name)[1]
    if file_extension == '.csv':
        data = pd.read_csv(file_name, encoding='gbk')
    elif file_extension == '.xlsx':
        data = pd.read_excel(file_name, encoding='gbk')
    drop_col = ['imei_', 'area_', 'msisdn_']
    data.drop(drop_col, axis=1, inplace=True)
    col = data.columns
    for each in col:
        if data[each].dtype == object:
            data[each] = data[each].str.replace('\t', '')
    print('data load successfully')
    return data

In [85]:
os.listdir('../Project Data/')

os.chdir('../Project Data/') # 切换到数据所在目录

['4月',
 '20190529',
 '20190529-4G电围.xls',
 '.DS_Store',
 '分析数据',
 '3月',
 'data_eda.ipynb',
 'same',
 '20190529-001.csv',
 '20190424-001.xlsx',
 '20190524-001.csv',
 '20190529-4G_resave.csv',
 'test.py',
 '20190422-灵瞳感知器.xls',
 '最新数据',
 '050823数据表']

In [87]:
data = get_data('20190524-001.csv')

data load successfully


In [88]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
0,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
1,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
3,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
4,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [89]:
day, week, month = get_frequency(data, treshold=2)

data processed successfully


In [90]:
day.head(15)

device_id_           DX-SZSC001  LT-SZSC001  YD-SZSC001
imsi_           day                                    
222992312514644 8             0           1           0
                9             0           2           0
                10            0           2           0
                11            0           2           0
                13            0           4           0
                14            0           3           0
                15            0           1           0
                16            0           5           0
                17            0           4           0
                20            0           1           0
                21            0           2           0
                22            0           2           0
                23            0           2           0
                24            0           3           0
228017227822524 17            0           0           3

In [52]:
day = day.reset_index(level=0)  # https://www.cnblogs.com/beyondChan/p/10926788.html

# multiple index 的方法

In [92]:
week.head(15) # 表示第19周，第20周出现的频率

device_id_            DX-SZSC001  LT-SZSC001  YD-SZSC001
imsi_           week                                    
222992312514644 19             0           7           0
                20             0          17           0
                21             0          10           0
228017227822524 20             0           0           3
228017228577400 19             0           3           0
                20             0           5           0
                21             0           1           0
234100485806791 21             0           1           1
234159176293646 21             0           0           2
234304160946400 20             0           4           0
234336539047034 19             0           3           0
250992214610153 19             0           3           0
272023115950103 20             0           2           0
302720509833859 20             0           1           0
                21             0           1           0